In [137]:
import pandas as pd
import sqlite3 as sq
from datetime import datetime
import numpy as np
ACF,AHRQ,ALLCDC,ATSDR,CGH,CDC,EPO,HIRP,LSPPPO,NCCDPHP,NCEZID,NCEH,NCHM,NCHS,NCHHSTP,NCIRD,NCIPC,CID,NCBDD,NIOSH,OSTLTS,OCPHP,OGDP,OHS,OID,OMH,ONDIEH,OPHPR,ODCDC,OSELS,OWH,OWCD,NCPHI,PHPPO,PHSPO,SEPDPO,HRSA,FDA,VA

In [138]:
pd.options.display.max_columns=999

In [139]:
con_nih = sq.connect('nih.db')

In [162]:
# only NIH projects
sql = '''
Select PI_IDS, PI_NAMEs, CORE_PROJECT_NUM, FY, ADMINISTERING_IC, TOTAL_COST, TOTAL_COST_SUB_PROJECT, ACTIVITY, FUNDING_MECHANISM
From projects
Where ADMINISTERING_IC in ('CC','RG', 'CIT', 'TW', 'TR', 'AT',
    'CA', 'RR', 'EY', 'HG', 'HL', 'AG', 'AA', 'AI', 'AR', 'EB', 'HD',
    'DA', 'DC', 'DE', 'DK', 'ES', 'GM', 'MH', 'MD', 'NS', 'NR', 'LM', 'OD' )
'''
df_projects = pd.read_sql(sql, con_nih)

In [163]:
df_projects.FUNDING_MECHANISM.unique()

array(['Non-SBIR/STTR RPGs', 'TRAINING, INDIVIDUAL', 'RESEARCH CENTERS',
       'OTHER RESEARCH-RELATED', 'TRAINING, INSTITUTIONAL',
       'SBIR-STTR RPGs', None, 'Contracts, Extramural', 'Construction',
       'Other Research Related', 'Research Projects',
       'Training, Individual', 'Research Centers', 'Unknown', 'SBIR-STTR',
       'Training, Institutional', 'Intramural Research', 'Other',
       'Non SBIR/STTR Contracts', 'INTRAMURAL RESEARCH',
       'INTERAGENCY AGREEMENTS', 'SBIR/STTR Contracts', 'OTHERS',
       'CONSTRUCTION GRANTS'], dtype=object)

In [167]:
# Missing FUNDING_MECHANISM in Earlier Years 
# So Use activity to filter data 
research_activities = df_projects.ACTIVITY[df_projects.FUNDING_MECHANISM.isin(['RESEARCH CENTERS','Research Projects', 'Research Centers'])].unique()

In [168]:
df_projects = df_projects[df_projects.ACTIVITY.isin(research_activities)]

In [170]:
df_projects['TOTAL_COST'] = df_projects[['TOTAL_COST']].where(~df_projects['TOTAL_COST'].isnull(),0)
df_projects['TOTAL_COST_SUB_PROJECT'] = df_projects[['TOTAL_COST_SUB_PROJECT']].where(~df_projects['TOTAL_COST_SUB_PROJECT'].isnull(),0)
df_projects['TOTAL_COST']  = df_projects['TOTAL_COST'] + df_projects['TOTAL_COST_SUB_PROJECT']
#df_projects = df_projects[df_projects.TOTAL_COST >= 50000]
#only look at 2001-2015
before_2001 = df_projects['FY'] < 2001
after_2015  = df_projects['FY'] > 2015
df_projects = df_projects[(~before_2001) & (~after_2015)]

#### Retrieve each indiviual pid

In [173]:
df_temp = df_projects[['PI_IDS','PI_NAMEs','CORE_PROJECT_NUM', 'FY', 'ADMINISTERING_IC', 'TOTAL_COST']]
df_temp.ADMINISTERING_IC.where(df_temp.ADMINISTERING_IC == 'GM', 'others', inplace=True)
df_temp.ADMINISTERING_IC.where(df_temp.ADMINISTERING_IC == 'others', 'GM', inplace=True)
df_temp.ADMINISTERING_IC.unique()

/Users/weizheng/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


array(['others', 'GM'], dtype=object)

In [174]:
#create pid and name pairs
vals = df_temp.values;
rows, cols = vals.shape
p_ids = []
p_names = []
p_nums = []
fys = []
ics = []
costs = []
#grab each pid from the ';' separated strings
for i in range(0,rows):
    if vals[i, 0] is not None and vals[i, 1] is not None:
        ids = vals[i, 0].strip().split(';')
        names = vals[i,1].strip().split(';')
        ids.pop()
        names.pop()
        if len(ids) == len(names):
            #if there are more than one pid, take primary contact
            if len(ids) == 1:
                ids = [d.strip() for d in ids ]
                names = [n.strip() for n in names]
            else:
                #ids = [d.replace(' (contact)', '').strip() for d in ids if '(contact)'   in d]
                #names = [n.replace(' (contact)', '').strip() for n in names if '(contact)' in n]
                ids = [d.replace(' (contact)', '').strip() for d in ids ]
                names = [n.replace(' (contact)', '').strip() for n in names]
            p_num = vals[i,2]
            fy = vals[i,3]
            ic = vals[i,4]
            cost = vals[i,5]
            p_ids.extend(ids)
            p_names.extend(names)
            p_nums.extend([str(p_num)] * len(ids))
            fys.extend([fy] * len(ids))
            ics.extend([ic] * len(ids))
            costs.extend([cost /  len(ids)] * len(ids))


In [175]:
data = {
    "pid": p_ids,
    "p_names": p_names,
    "p_nums": p_nums,
    "fys" : fys,
    "ics" : ics,
    "costs" : costs
}
df_temp = pd.DataFrame(data)

In [176]:
df_temp.shape

(903129, 6)

In [177]:
df_cost = df_temp[['pid', 'fys', 'costs']]
df_pid_costsum_by_year = df_cost.groupby(['pid','fys']).sum()
df_pid_costsum_by_year.reset_index(inplace=True)
df_pid_costsum_by_year = df_pid_costsum_by_year[df_pid_costsum_by_year.costs >= 50000]
df_pid_costsum_by_year = df_pid_costsum_by_year[df_pid_costsum_by_year.pid !='']

In [178]:
df_pid_costsum_by_year.shape

(467505, 3)

In [179]:
df_pid_costsum_by_year.head(5)

,pid,fys,costs
4,10000396,2010,125610.0
5,10000396,2011,142429.0
6,10000396,2012,154636.0
7,10000396,2013,151791.0
8,10000396,2014,128800.0


In [180]:
df_temp.head(5)

,costs,fys,ics,p_names,p_nums,pid
0,174299.0,2015,others,"LIU, SHAN-LU",R21AI109464,10637051
1,25000.0,2014,others,"NAJJAR, SONIA M.",R01DK054254,1936803
2,7082.5,2015,others,"NAVAS-ACIEN, ANA",R01ES021367,8696712
3,7082.5,2015,others,"VAIDYA, DHANANJAY MADHUKAR",R01ES021367,8656693
4,349856.0,2015,others,"WILLIAMS, TYISHA",R15NS084329,14584389


#### Take out researchers who have supports less than $50000 per year

In [181]:
df_temp = df_temp[df_temp.pid.isin(df_pid_costsum_by_year.pid)]

In [182]:
p_ids = list(df_temp.pid)
p_names = list(df_temp.p_names)
p_nums = list(df_temp.p_nums)
fys = list(df_temp.fys)
ics = list(df_temp.ics)

In [183]:
def getUniqDF(arrayList):
    #taking equal-length arrays, create  unique combinations
    cs = ['_'.join([str(a) for a in A]) for A in zip(*arrayList)]
    uniq_comb = list(set(cs))
    a_len = len(arrayList)
    outputs = [];
    for i in range(a_len):
        outputs.append([])
    #unique combinations
    for c in uniq_comb:
        s = c.split('_')
        for si in range(a_len):
            outputs[si].append(s[si])
    return(outputs)
# pid and names
uniq_pids, uniq_names = getUniqDF([p_ids, p_names])
df_pid_name = pd.DataFrame({'pid': uniq_pids, 'full_name':uniq_names})

# pids and project number
pid4num, uniq_prjnum = getUniqDF([p_ids, p_nums])
df_pid_prjnum = pd.DataFrame({'pid': pid4num, 'project_key':uniq_prjnum})

# pids, fy, and ic
pid4fy, fys_support, ics_support = getUniqDF([p_ids, fys, ics])   
df_pid_fy_ic = pd.DataFrame({'pid': pid4fy, 'fy':fys_support, 'ic': ics_support })



In [189]:
#calculate years of support 
df_pid_fy_ic['val'] = 1
df_sum = df_pid_fy_ic[['pid', 'ic', 'val']]
df_support = df_sum.pivot_table(columns='ic', index='pid', values='val', aggfunc='sum')
df_support.columns = ['nigms_years', 'others_years']
df_support['nih_years'] =  df_support[['nigms_years', 'others_years']].apply(np.max, axis=1)



In [190]:
df_support.head(5)

,nigms_years,others_years,nih_years
pid,,,
10000396,NaN,5.0,5.0
10000490,NaN,2.0,2.0
10001078,NaN,3.0,3.0
10001128,NaN,1.0,1.0
10001159,NaN,1.0,1.0


### NIGMS researchers

In [191]:
#definition of continually supported
nigms_inds = df_support.nigms_years>1
nigms_contin_inds = df_support.nigms_years==15
nih_contin_inds = (df_support.nih_years==15) & (df_support.nigms_years > 1)
df_support['is_study'] = 0
df_support['is_comp'] = 0
df_support['is_nigms'] = 0
df_support.ix[nigms_contin_inds, 'is_study' ] = 1
df_support.ix[nih_contin_inds & (~nigms_contin_inds), 'is_comp' ] = 1
df_support.ix[nigms_inds, 'is_nigms'] = 1

In [192]:
df_support[nigms_inds].index

Index(['10008168', '10010745', '10010983', '10011950', '10021299', '10028321',
       '10036621', '10039101', '10058046', '10058461',
       ...
       '9971733', '9974889', '9983921', '9984179', '9985280', '9987162',
       '9996634', '9996881', '9997153', '9997198'],
      dtype='object', name='pid', length=8665)

In [193]:
len(df_support[nigms_inds]) 

8665

#### Continually Supported by NIGMS

In [197]:
len(df_support.ix[nigms_contin_inds, 'is_study' ])

523

#### Researchers who were continually supported by NIH

In [198]:
len(df_support.ix[nih_contin_inds & (~nigms_contin_inds), 'is_comp' ])

458

### Compare with Andrew pids

In [207]:
andrew_df.iloc[:,0]

0      1852587
1      1857694
2      1858017
3      1858018
4      1858025
5      1858106
6      1858378
7      1858379
8      1858393
9      1858403
10     1858404
11     1858415
12     1858679
13     1858704
14     1858903
15     1858963
16     1859098
17     1859360
18     1859365
19     1859377
20     1859694
21     1860024
22     1860213
23     1860225
24     1860231
25     1860244
26     1860696
27     1860709
28     1860711
29     1860974
        ...   
914    1902025
915    1928546
916    1938841
917    2076451
918    6214835
919    1857799
920    1858913
921    1860214
922    1860612
923    1860763
924    1862872
925    1863274
926    1863500
927    1864449
928    1877969
929    1882172
930    1883032
931    1883263
932    1884515
933    1885202
934    1885388
935    1889277
936    1896344
937    1896522
938    1897376
939    1901968
940    1946799
941    2072820
942    6611541
943    7354240
Name: ﻿PI_IDS, dtype: int64

In [211]:
study_set = set(df_support.ix[nigms_contin_inds, 'is_study' ].index)
comp_set = set(df_support.ix[nih_contin_inds & (~nigms_contin_inds), 'is_comp' ].index) 
whole_set = study_set | comp_set
andrew_df = pd.read_csv('./data/andrew_pids.csv', header =0)
andrew_df = andrew_df.applymap(str)
andrew_set = set(andrew_df.iloc[:,0].values)

In [212]:
our_notin_andrew = (whole_set - andrew_set)
len(ours_notin_andrew)

272

In [214]:
andrew_notin_ours = andrew_set - whole_set
len(andrew_notin_ours)

235

In [215]:
join_set = whole_set & andrew_set
len(join_set)

709

### Create Author Table

In [216]:
df_pid_name.drop_duplicates(['pid'], inplace=True)
df_pid_name = df_pid_name[~df_pid_name.pid.isnull()]
df_pid_name = df_pid_name[df_pid_name.full_name != 'WEDEN, MARGARET']

In [217]:
temp_df = df_pid_name.copy()
temp_df['pi_key'] = 'nih'+'_'+ temp_df['pid']
temp_df['full_name'] = temp_df['full_name'].str.lower()
temp_df['last_name'] = temp_df['full_name'].str.split(',').str.get(0)
temp_df['f_m_name'] = temp_df['full_name'].str.split(',').str.get(1)
temp_df['f_m_name'] = temp_df['f_m_name'].str.strip()
temp_df['first_name'] = temp_df['f_m_name'].str.split(' ').str.get(0)
temp_df['middle_name'] = temp_df['f_m_name'].str.split(' ').str.get(1)
temp_df['last_initial'] = temp_df['last_name'].str[0]
temp_df['first_initial'] = temp_df['first_name'].str[0]
temp_df['middle_initial'] = temp_df['middle_name'].str[0]



In [219]:
temp_df.shape

(79763, 10)

In [220]:
temp_df = temp_df.merge(df_support[['is_study', 'is_comp']], left_on='pid', right_index=True)
temp_df.drop(['f_m_name', 'pid'], axis=1, inplace=True)

In [222]:
len(temp_df[temp_df.is_comp == 1])

458

In [224]:
temp_df.head(5)

,full_name,pi_key,last_name,first_name,middle_name,last_initial,first_initial,middle_initial,is_study,is_comp
0,"weisman, lois s",nih_1881574,weisman,lois,s,w,l,s,1,0
1,"herman, tory g",nih_1952753,herman,tory,g,h,t,g,0,0
2,"willcox, donald craig",nih_8942365,willcox,donald,craig,w,d,c,0,0
3,"van der water, judy",nih_7004345,van der water,judy,NaN,v,j,NaN,0,0
4,"lombard, julian h",nih_3318913,lombard,julian,h,l,j,h,0,0


In [225]:
#create an author table
con_analysis_db = sq.connect('nih_analyses.db')

In [226]:
temp_df.to_sql('researcher',con_analysis_db, if_exists='replace')
con_analysis_db.execute('create unique index pi_key_index on researcher(pi_key);')

In [227]:
df_author = temp_df

### Create Author and Project Link table

In [228]:
temp_df = df_pid_prjnum.copy()
temp_df['pi_key'] = 'nih' + '_' + temp_df['pid']
temp_df.drop('pid', axis=1, inplace=True)
temp_df.to_sql('res_prj',con_analysis_db, if_exists='replace')
con_analysis_db.execute('create index pi_key_index_res_prj on res_prj(pi_key);')
con_analysis_db.execute('create index prjnum_index_res_prj on res_prj(project_key);')

In [229]:
temp_df.head(5)

,project_key,pi_key
0,R01CA193994,nih_6377535
1,P01HL060898,nih_8227165
2,R01HG004962,nih_7612243
3,R01HL092121,nih_2787948
4,M01RR000827,nih_1876331


In [234]:
df_study = df_author[df_author.is_study == 1]
df_comp = df_author[df_author.is_comp == 1]
df_study = df_study.merge(temp_df, on='pi_key', how='left') 
df_comp = df_comp.merge(temp_df, on='pi_key', how='left') 

### Create Project and Publication Link table

In [134]:
sql = '''
Select * 
From link
'''

df_prj_pub_link = pd.read_sql(sql, con_nih)

In [135]:
df_prj_pub_link.columns = ['publication_key' , 'project_key']

In [136]:
df_prj_pub_link.to_sql('prj_pub',con_analysis_db, if_exists='replace')
con_analysis_db.execute('create index prjnum_index_prj_pub on prj_pub(project_key);')
con_analysis_db.execute('create index pubkey_index_prj_pub on prj_pub(publication_key);')

In [239]:
df_study = df_study.merge(df_prj_pub_link, on='project_key', how='left')
df_comp = df_comp.merge(df_prj_pub_link, on='project_key', how='left')

In [241]:
df_study.head(5)

,full_name,pi_key,last_name,first_name,middle_name,last_initial,first_initial,middle_initial,is_study,is_comp,project_key,publication_key
0,"weisman, lois s",nih_1881574,weisman,lois,s,w,l,s,1,0,R37GM062261,15821138.0
1,"weisman, lois s",nih_1881574,weisman,lois,s,w,l,s,1,0,R37GM062261,15684027.0
2,"weisman, lois s",nih_1881574,weisman,lois,s,w,l,s,1,0,R37GM062261,16437158.0
3,"weisman, lois s",nih_1881574,weisman,lois,s,w,l,s,1,0,R37GM062261,18653471.0
4,"weisman, lois s",nih_1881574,weisman,lois,s,w,l,s,1,0,R37GM062261,18391069.0


### Join Author with Publications

In [242]:
sql = '''
Select * 
From publication
'''
df_pub = pd.read_sql(sql, con_nih)


In [243]:
df_pub.head(5)

,AFFILIATION,AUTHOR_LIST,COUNTRY,ISSN,JOURNAL_ISSUE,JOURNAL_TITLE,JOURNAL_TITLE_ABBR,JOURNAL_VOLUME,LANG,PAGE_NUMBER,PMC_ID,PMID,PUB_DATE,PUB_TITLE,PUB_YEAR
0,None,"Rudnisky, Christopher J; Belin, Michael W; Guo...",United States,1879-1891,None,American journal of ophthalmology,Am J Ophthalmol,162,eng,89-98.e1,NaN,26550696,2016 Feb,Visual Acuity Outcomes of the Boston Keratopro...,2016
1,None,"Stanton, Bonita; Dinaj-Koci, Veronica; Wang, B...",United States,1573-3254,6,AIDS and behavior,AIDS Behav,20,eng,1182-96,4842173.0,26499123,2016 Jun,Adolescent HIV Risk Reduction in the Bahamas: ...,2016
2,None,"Phipps, Matthew C; Huang, YiHui; Yamaguchi, Ry...",United States,1554-527X,2,Journal of orthopaedic research : official pub...,J Orthop Res,34,eng,307-13,NaN,26016440,2016 Feb,In vivo monitoring of activated macrophages an...,2016
3,None,"Herrera-Almario, Gabriel E; Kirk, Katherine; G...",United States,1879-1883,2,American journal of surgery,Am J Surg,211,eng,315-20,4789170.0,26590043,2016 Feb,The effect of video review of resident laparos...,2016
4,None,"Baker, Marissa G; Stover, Bert; Simpson, Chris...",Germany,1432-1246,4,International archives of occupational and env...,Int Arch Occup Environ Health,89,eng,679-87,4829443.0,26589320,2016 May,Using exposure windows to explore an elusive b...,2016


In [245]:
df_study = df_study.merge(df_pub, left_on='publication_key',  right_on='PMID', how='left')
df_comp = df_comp.merge(df_pub, left_on='publication_key',  right_on='PMID', how='left')

In [248]:
df_study.to_csv('./data/study_pub.csv', header=True, index=False)
df_comp.to_csv('./data/study_pub.csv', header=True, index=False)